In [1]:
import tensorflow as tf
import sys
sys.path.append("..")
import tools
import matplotlib.pyplot as plt
from astroML.crossmatch import crossmatch_angular
import numpy as np
from lsst.daf.butler import Butler
import pandas as pd
import multiprocessing
import os
%load_ext autoreload
%autoreload 2

2024-04-13 03:57:35.749976: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 03:57:35.750027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 03:57:35.750858: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 03:57:35.757147: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 03:57:39.608302: W tensorflow/compiler/tf2

In [36]:
tfrecord_path = "/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/test1.tfrecord"
val_index_path="/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/val_index1.npy"
repo_path ="/epyc/ssd/users/kmrakovc/DATA/rc2_subset/SMALL_HSC/"
collection = "u/kmrakovc/single_frame_injection_02"
model = "../DATA/Trained_model1"
with open(val_index_path, 'rb') as f:
    val_index = np.load(f)
    val_index.sort()

In [37]:
from lsst.daf.butler import Butler
butler = Butler(repo_path)
catalog_ref = list(butler.registry.queryDatasets("injected_postISRCCD_catalog",
                                                     collections=collection,
                                                     instrument='HSC'))
injected_calexp_ref = list(butler.registry.queryDatasets("injected_calexp",
                                             collections=collection,
                                             instrument='HSC'))
calexp_ref = list(butler.registry.queryDatasets("calexp",
                                             collections=collection,
                                             instrument='HSC'))

In [38]:
i = val_index[0]
injected_calexp = butler.get("injected_calexp",
                                 dataId=injected_calexp_ref[i].dataId,
                                 collections=collection)
calexp = butler.get("calexp",
                    dataId=calexp_ref[i].dataId,
                    collections=collection)
catalog = butler.get("injected_postISRCCD_catalog",
                         dataId=catalog_ref[i].dataId,
                         collections=collection)


In [39]:
start = calexp.wcs.pixelToSky(0 ,0)
end = calexp.wcs.pixelToSky(calexp.getDimensions()[0], calexp.getDimensions()[1])

In [40]:
calexp.getDimensions()[0], calexp.getDimensions()[1]

(2048, 4176)

In [41]:
start.getRa(), start.getDec(), end.getRa(), end.getDec(), 

(Angle(150.40105702664883, degrees),
 Angle(2.1787852330284645, degrees),
 Angle(150.20551894755198, degrees),
 Angle(2.0826792661680118, degrees))

In [42]:
origin_calexp = calexp.getWcs().skyToPixelArray(np.array([catalog["ra"]]), np.array([catalog["dec"]]),
                                                 degrees=True)
origin_injected_calexp = injected_calexp.getWcs().skyToPixelArray(np.array([catalog["ra"]]), np.array([catalog["dec"]]),
                                                 degrees=True)

In [48]:
catalog[catalog["injection_flag"]==0]

injection_id,injection_flag,injection_draw_size,ra,dec,source_type,trail_length,mag,beta,visit
int64,int64,int64,float64,float64,str5,int64,float64,float64,int64
171,0,44,150.2307805831983,2.1036396260277863,Trail,17,20.76765625663676,62.635660756196,1242
172,0,44,150.35593550794263,2.134652524185544,Trail,17,23.748446306105862,41.24238387031755,1242
173,0,146,150.29329695510796,2.164501500051809,Trail,71,26.962597208312552,66.05030543599453,1242
174,0,124,150.22447306315473,2.148608501325771,Trail,60,22.78925880893416,30.379461896140775,1242
175,0,118,150.35964712901264,2.106219295509819,Trail,57,20.11706926931431,145.8012753033758,1242
176,0,82,150.3197901284553,2.1010682616603984,Trail,38,23.97096194767152,169.96700070639238,1242
177,0,66,150.3100503629959,2.1142644658520964,Trail,30,25.013708249785985,54.10656188214591,1242
178,0,124,150.27143899501453,2.177112472402571,Trail,60,22.81654037224018,110.57554644911244,1242
179,0,112,150.30686760237307,2.170456281168106,Trail,54,25.13298277394862,74.48031194581128,1242


In [44]:
origin_calexp

(array([1599.88955016,  942.10869601,  301.83168007,  638.84417736,
        1550.45387958, 1658.45659463, 1375.75447933,   32.07599416,
         175.09547506, 1790.59254535,  436.56600817, 1865.84191445,
         194.40922791,  776.28058378, 1559.97094688, 1225.09821678,
        1303.54911276, 1878.08772681,  158.60592898, 1030.36043262,
         -84.4138253 ,  -96.67768478]),
 array([3636.9014322 ,  965.2920732 , 2303.09434367, 3770.65948531,
         885.45249761, 1737.32155493, 1945.51165283, 2769.42272626,
        2013.6031667 , 4034.08087022,  917.09219486, 1216.29307025,
        2390.26728178, 2372.5375418 , 2668.94038774, 2533.74623192,
        2693.04559636,  284.2359307 , 2516.64460464, 1976.3149942 ,
        2848.49427212,  193.21943921]))

In [45]:
origin_injected_calexp

(array([1599.88949317,  942.10878933,  301.83165011,  638.84411807,
        1550.45402574, 1658.45663808, 1375.75449103,   32.0759189 ,
         175.09543654, 1790.59254966,  436.56603158, 1865.84204635,
         194.40918332,  776.28056663, 1559.97089835, 1225.09817724,
        1303.54906092, 1878.08800122,  158.60587658, 1030.36044331,
         -84.41392197,  -96.677934  ]),
 array([3636.90142481,  965.29230423, 2303.09425225, 3770.6594253 ,
         885.45254338, 1737.32155946, 1945.51170299, 2769.42260226,
        2013.60298924, 4034.08089194,  917.09229577, 1216.29294249,
        2390.26715336, 2372.53755954, 2668.94043474, 2533.74625929,
        2693.04561784,  284.23571153, 2516.64447852, 1976.31505894,
        2848.49412751,  193.21915815]))

In [34]:
origin_calexp[0].shape

(21,)